In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact

In [ ]:
pip install matplotlib

In [ ]:
# Read the Mapbox API key
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path('sfo_neighborhoods_census_data.csv')
sf_data = pd.read_csv(file_path, index_col="year")

file_path_2 = Path('neighborhoods_coordinates.csv')
df_neighborhood_locations = pd.read_csv(file_path)

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""

    
    housing_units = sf_data["housing_units"]
    housing_units = housing_units.groupby([sf_data.index]).mean()
    
    housing_units_fig = plt.figure()
    housing_units_plot = housing_units.plot.bar(x='year', ylim=[1000, 1000], title="Housing Units per Year")
    plt.close(housing_units_fig)
    
    return pn.pane.matplotlib(housing_units_fig)

In [ ]:
def average_gross_rent():  
    """Average Gross Rent in San Francisco Per Year."""
    average_rent_sqr_foot = sf_data["gross_rent"].groupby([sf_data.index]).mean()
    
    gross_rent_fig = plt.figure()
    average_rent_plot = (average_rent_sqr_foot.plot.line(x='year', y='gross_rent', title="Average Gross Rent per Year"))
    plt.close(gross_rent_fig)
    
    return pn.pane.Matplotlib(gross_rent_fig)

In [ ]:
def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_price_sqr_foot = sf_data["sale_price_sqr_foot"].groupby([sf_data.index]).mean()
    sales_price_fig = plt.figure()
    average_price_plot = (average_price_sqr_foot.plot.line(x='year', y='sale_price_sqr_foot', title="Average Sales Price per Year"))
    plt.close(sales_price_fig)
    
    return pn.pane.Matplotlib(sales_price_fig)

In [ ]:
def average_price_by_neighborhood(neighborhood):
    """Average Prices by Neighborhood."""
    
    sf_data_new = sf_data.groupby([sf_data.index, "neighborhood"]).mean()
    sfo_sales = sf_data_new["sale_price_sqr_foot"]
    sfo_sales_df = pd.DataFrame(sfo_sales).reset_index()
     
    
    return sfo_sales_df.loc[sfo_sales_df['neighborhood']==neighborhood,:].hvplot.line(
        x="year",
        y="sale_price_sqr_foot",
        colormap="coolwarm",
        title="SF Sale per Square foot per Year",
    )

sf_data_new = sf_data.groupby([sf_data.index, "neighborhood"]).mean()
sfo_sales = sf_data_new["sale_price_sqr_foot"]
sfo_sales_df = pd.DataFrame(sfo_sales).reset_index()
neighborhood_choice = sfo_sales_df["neighborhood"].unique()
interact(average_price_by_neighborhood, neighborhood=neighborhood_choice)

In [ ]:
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    avg_value_per_neighborhood = sf_data.groupby([sf_data["neighborhood"]]).mean()
    avg_value_per_neighborhood = avg_value_per_neighborhood.reset_index()
    top_10_most_expensive = avg_value_per_neighborhood.nlargest(10, 'sale_price_sqr_foot').reset_index()
    top10_plot = px.bar(top_10_most_expensive,
    x="neighborhood", 
    y="sale_price_sqr_foot", 
    title="Top 10 Most Expensive Neighborhoods", 
    )

    return top10_plot

In [ ]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    avg_value_per_neighborhood = sf_data.groupby([sf_data["neighborhood"]]).mean()    
    top_10_most_expensive = avg_value_per_neighborhood.nlargest(10, 'sale_price_sqr_foot').reset_index()
    top_10_parallel = px.parallel_coordinates(
    top_10_most_expensive, 
    color='sale_price_sqr_foot')

    return top_10_parallel

In [ ]:
def parallel_categories():
    """Parallel Categories Plot."""
    
    avg_value_per_neighborhood = sf_data.groupby([sf_data["neighborhood"]]).mean()  
    top_10_ex_neighborhood = avg_value_per_neighborhood.nlargest(10, 'sale_price_sqr_foot').reset_index()
    top_10_parallel_categories = px.parallel_categories(
    top_10_ex_neighborhood,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",)

    return top_10_parallel_categories

In [ ]:
def neighborhood_map():
    """Neighborhood Map"""
    
    file_path = Path('neighborhoods_coordinates.csv')
    df_neighborhood_location = pd.read_csv(file_path)
    avg_value_per_neighborhood = sf_data.groupby([sf_data["neighborhood"]]).mean()  
    avg_value_per_neighborhood = avg_value_per_neighborhood.reset_index()
    avg_value_location = pd.concat([avg_value_per_neighborhood, df_neighborhood_location], axis="columns", join="inner")

    location_map = px.scatter_mapbox(
    avg_value_location,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent")
    
    return location_map

In [ ]:
def sunburst():
    """Sunburst plot."""
    fig = px.sunburst(sf_data,
                  path=["neighborhood", "gross_rent", "sale_price_sqr_foot"],
                  values='year',
                  width=800, height=500)
    fig.show() 
    

In [ ]:
#Create a Title for the Dashboard
title = '#San Francisco Real Estate Analysis from 2010-2016'
presentation_tab = pn.Row(pn.Column(title), neighborhood_map())

#Create a tab layout for the Dashboard
market_analysis_row = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())


neighborhood_analysis_tab = pn.Column(interact(average_price_by_neighborhood, neighborhood=neighborhood_choice),
    top_most_expensive_neighborhoods())

parallel_plots_tab = pn.Column( 
    parallel_categories(),
    parallel_coordinates())


# Create tabs

SF_REPrices_Dashboard = pn.Tabs(("Real Estate Presentation", presentation_tab), 
("Yearly Market Analysis", market_analysis_row),
("Neighborhood Analysis", neighborhood_analysis_tab), 
("Interactive Market Analysis by Neighborhood", parallel_plots_tab))

In [ ]:
#Serve the Dashboard
dashboard = pn.Column(SF_REPrices_Dashboard, width=900)

In [ ]:
dashboard.servable()